# Método de Taylor de integración de EDOs

En el notebook [anterior](https://github.com/dpsanders/metodos_numericos_garantizados/blob/master/notebooks/8.%20Series%20de%20Taylor.ipynb) hicimos varios ejercicios relacionados con desarrollos de Taylor de funciones suaves (de una variable), y cómo construir éstas para las funciones elementales. En este notebook, extenderemos esas ideas para construir la solución de un problema de valor inicial (ecuación diferencial ordinaria con condición inicial).

En particular, en el último ejercicio, demostraron que, dado el problema de valor inicial

\begin{equation}
\dot{x} = f(t,x), \quad x(0)=x_0,
\end{equation}

entonces, *la solución* $x(t)$ de esta ecuación, para tiempos suficientemente cercanos a $t_0=0$, la podemos escribir como:

\begin{equation}
x(t) = \sum_{k=0} x_{[k]}(t_0) \; (t-t_0)^k,
\end{equation}

donde los coeficientes del desarrollo satisfacen la relación de recurrencia:

$$ x_{[n+1]}(t_0) = \frac{f_{[n]}(t_0)}{n+1}, $$

y $f_{[n]}(t_0)$ son los coeficientes del desarrollo en serie de Taylor de $f(t,x)$ alrededor de $t_0=0$.

**[1]** Enuncia el teorema de existencia y unicidad de la solución del problema de valor inicial.

Sea  $f(t, x):\Omega\subseteq\mathbb{R}\times\mathbb{R}^n\longrightarrow\mathbb{R}^n$, donde $\Omega $ es abierto, una función continua y localmente Lipschitz respecto de 
$x$ (interprétese $f(t, x)$ como la forma estándar de una EDO n-dimensional de primer orden). Entonces, dado $Ω(t_{0}, x_{0}) \in \Omega$, podemos encontrar un intervalo cerrado  $I_{\alpha}=[t_{0}-\alpha, t_{0}+\alpha]\subset \mathbb{R}, \alpha \in \mathbb{R}$ donde existe solución única del siguiente problema de Cauchy:

$$\begin{cases}x'=f(t, x) \\ x(t_{0})=x_{0}\end{cases}$$

que cumple que los pares $(t, x(t)) \in \Omega, \forall t \in I_{\alpha}$.

**[2]** Considera la ecuación diferencial y la condición inicial siguientes:

$$ \dot{x} = x^2, \quad x(0)=3. $$

(a) ¿Cuál es la solución analítica ("exacta") de esta ecuación?

* $x(t) = -\dfrac{1}{t + c}$, de la condición inicial inicial tenemos $c = -\frac{1}{3}$

(b) ¿Existe la solución $x(t)$ para todo $t\in\mathbb{R}$?

* No, en este caso no exite para $t = \frac{1}{3}$

(c) Partiendo de la condición inicial, construye explícitamente la solución en serie de Taylor de la ecuación anterior, digamos hasta orden 3. Esto lo haremos sustituyendo la aproximación que tenemos de la solución (que inicialmente es $x(t)\rightarrow x_0$) en la ecuación diferencial, y obteniendo sucesivamente los coeficientes de orden superior de la serie.

* $x_{[0]} = x_0 = 3$
* $x_{[1]} = (x(t))^2 = (x_0)^2 $
* $2x_{[2]}t = (x(t))^2 = (x_0 + x_0^2 t)^2 = 2x_0x_0^2 t \implies x_{[2]} = x_0^3 $
* $3x_{[3]}t^2 = (x(t))^2 = (x_0 + x_0^2 t + x_0^3 t^2)^2 = x_0^4t^2 + 2x_0x_0^3 t^2 \implies x_{[3]} = x_0^4 $
* $4x_{[4]}t^3 = (x(t))^2 = (x_0 + x_0^2 t + x_0^3 t^2 + x_0^4 t^3)^2 = 2x_0^2x_0^3t^3 + 2x_0x_0^4 t^3  \implies x_{[4]} = x_0^5 $

(d) Implementa el punto anterior en la computadora, usando la estructura que definiste para las series de Taylor en el notebook anterior.



In [1]:
mutable struct Taylor
    orden::Int64
    coef::Array{Float64, 1}
    Taylor(coef) = new(length(coef)-1, coef)
end

import Base: +, -, *, /,^, getindex, setindex!,==

function getindex(f::Taylor, i::Int64)
    f.coef[i + 1]
end

function getindex(f::Taylor, r::UnitRange{Int64})
    newcoef = f.coef[(r[1] +1): (r[end] +1)]
    Taylor(newcoef)
end

function setindex!(f::Taylor, x::Float64, i::Int64)
    f.coef[i + 1] = x
end

function setindex!(f::Taylor, x::Int64, i::Int64)
    f.coef[i + 1] = x
end

function ==(f::Taylor, g::Taylor)
    f.orden == g.orden && f.coef == g.coef  ? true : false
end

function +(f::Taylor, g::Taylor)
    if f.orden == g.orden
        Taylor(f.coef .+ g.coef)
    elseif f.orden < g.orden
        Taylor(f.coef[1:f.orden + 1] .+ g.coef[1:f.orden + 1])
    else
        Taylor(g.coef[1:g.orden + 1] .+ f.coef[1:g.orden + 1])
    end
end

function -(f::Taylor, g::Taylor)
    if f.orden == g.orden
        Taylor(f.coef .- g.coef)
    elseif f.orden < g.orden
        Taylor(f.coef[1:f.orden + 1] .- g.coef[1:f.orden + 1])
    else
        Taylor(-g.coef[1:g.orden + 1] .+ f.coef[1:g.orden + 1])
    end
end

function *(f::Taylor, g::Taylor)
    f.orden != g.orden && error("Ordenes distintos")
    newcoef = Array{Float64, 1}(f.orden + 1)
    for k in 0:f.orden 
        Taylori = 0.0
        for i in 0:k
            Taylori += f[i]*g[k - i]  
        end
        newcoef[k + 1] = Taylori 
    end
    Taylor(newcoef)
end

function /(f::Taylor, g::Taylor)
    f.orden != g.orden && error("Ordenes distintos")
    newcoef = Array{Float64, 1}(f.orden + 1)
    
    for k in 0:f.orden 
        Taylori = f[k]
        for i in 0:k - 1
            Taylori -= (f[i]/g[i])*g[k - i]  
        end
        newcoef[k + 1] = Taylori/g[0] 
    end
    Taylor(newcoef)
end


function ^(f::Taylor, n::Int64)
    newTaylor = f
    for i in 1:n-1
        newTaylor *= f
    end
    newTaylor
end

^ (generic function with 53 methods)

In [2]:
function serieTaylor(f,  x0::Float64, orden::Int64=10)
    newTaylor = Taylor(zeros(orden + 1))
    newTaylor[0] = x0
    newTaylor[1] = f(x0)
    for i in 2:orden
        x0 = newTaylor[0:i]
        newTaylor[i] = (f(x0)[i-1])/(i)
    end
    newTaylor
end

serieTaylor (generic function with 2 methods)

In [3]:
f(x) = x^2

f (generic function with 1 method)

In [4]:
T2 = serieTaylor(f, 3.0)

Taylor(10, [3.0, 9.0, 27.0, 81.0, 243.0, 729.0, 2187.0, 6561.0, 19683.0, 59049.0, 177147.0])

In [5]:
(177147.0)*(1/3^10)

3.0

In [6]:
g(x) = x^3

g (generic function with 1 method)

In [7]:
T = serieTaylor(g, 2.0)

Taylor(10, [2.0, 8.0, 48.0, 320.0, 2240.0, 16128.0, 118272.0, 878592.0, 6.58944e6, 4.97869e7, 3.7838e8])

In [8]:
9/2

4.5

In [9]:
t = 1/9
xt = 0
n = T2.orden
for i in 0:n
    xt += T2[i]*t^i
    if i == n 
        @show T2[i]*t^i
    end
        
end
xt

T2[i] * t ^ i = 5.0805263425290837e-5


4.499974597368288

**[3]** (Paso de integración.) 

Consideremos que hemos obtenido una aproximación en serie de la solución de un problema de valor inicial hasta orden $P$. Suponiendo que $P$ es suficientemente grande para que podeamos suponer que estamos en la *cola convergente* de la serie, entonces *impondremos* que el último término de la serie obtenida sea, en valor absoluto menor, que $\epsilon_\textrm{abs}$. 

(a) Muestra que estas suposiciones permiten obtener un paso de integración.


* Si turncamos el polimonio a orden $p$ tenemos $\mid  x_{[p]} (t -t_0)^p\mid \leq \epsilon_{abs}$ entonces el paso de integración lo podemos obtener $h = t - t_0 = \left( \dfrac{\epsilon _{abs}}{\mid x_{[p]}(t_0) \mid}  \right)^{1/p}$.

(b) ¿Qué pasa con el paso de integración (aumenta o disminuye) si aumentamos el orden de la serie?

* Si aumentamos el orden $x_{[p]}$ incrementa y el cociente disminuye y al calcularte la raíz p-ésima esto crece.
 

* En la pruebas hechas abajo, primero calculamos el para para un polinimio de orde $32$ cuyo paso fue de $3.4839848437617644$, luego se aumento el grado del polinomio a $64$ y el paso incremento a $5.902528986596986$.

(c) ¿Es el paso de integración fijo (constante) o varía? Argumenta.

* El paso varía pues depende del grado del polinomio y del punto donde se haga la expansión 

(d) Implementa una función que determine el paso de integración y úsala para la ecuación diferencial del ejercicio **2**.

In [63]:
eps()

2.220446049250313e-16

In [76]:
function paso(x::Taylor)
    p = length(x.coef) - 1
    (1.0e-32/(abs(x[p])))^(1/p)
end

paso (generic function with 1 method)

In [77]:
T32 = serieTaylor(f, 0.1, 2^5)

@show paso(T32)

## Veamos con el paso de tiempo si aumentamos el orden del polinomio

T64 = serieTaylor(f, 0.1, 2^6)

@show paso(T64);

paso(T32) = 1.0746078283213174
paso(T64) = 3.2781211513934583


**[4]** 

(a) Implementa una función que integra lo que hemos hecho hasta ahora. Esto es, que genere el desarrollo local de la serie de Taylor (ver ejercicio **2**(d)), que use esta serie para obtener el paso de integración $h$, y que usando ambos, genere la *nueva* condición inicial al tiempo $t_0+h$. Para el último punto, usa el método de Horner (pregúntale a google) para sumar la serie.

(b) Usa tu función con el problema e integra varios pasos de integración. ¿Puedes integrar más allá de $t=1/3$?

* No pues la expasión en serie de Taylor es continua y la solución a la ecuación tiene una discontinuidad en $t = 1/3$ 


In [78]:
function horner(P::Taylor, x::Real)
    result = 0.0
    for i in P.orden:-1:0
        result = (result * x) + P[i]
    end
    result
end

horner (generic function with 1 method)

In [79]:
f(x) = x^2 
T22 = serieTaylor(f, 3.0, 33)

@show horner(T22, 1/9)

sf(x) = -1/(x - 1/3)

@show sf(1/9);

horner(T22, 1 / 9) = 4.5
sf(1 / 9) = 4.5


In [80]:
function serieTaylor2(f,  x0::Real, t0::Real, orden::Int64=10)
    newTaylor = Taylor(zeros(orden + 1))
    newTaylor[0] = x0
    newTaylor[1] = f(x0, t0)
    for i in 2:orden
        x0 = newTaylor[0:i]
        newTaylor[i] = (f(x0, t0)[i-1])/(i)
    end
    newTaylor
end

serieTaylor2 (generic function with 2 methods)

In [81]:
function taylorIntegration(f, x0::Real, t0::Real ,orden::Int64=32)
    T = serieTaylor2(f, x0, t0, orden)
    t = paso(T)
    x = horner(T, t)
    x, t + t0
end

taylorIntegration (generic function with 2 methods)

In [82]:
f(x, t) = x^2
x0 = 3.0
taylorIntegration(f, x0, 0.1)

(3.3208804044156417, 0.13220836699297975)

In [85]:
t = 0.0
x = 3.0
f(x, t) = x^2
for i in 1:150
    x, t = taylorIntegration(f, x, t, 32)
    #@show x, t 
    #@show (sf(t)-x)/sf(t)
end
@show x, t 
@show (sf(t)-x)/sf(t)

(x, t) = (715891.3328014722, 0.3333319364732873)
(sf(t) - x) / sf(t) = -8.327284396471174e-11


-8.327284396471174e-11

**[5]** 

(a) Muestra que la solución del problema de valor inicial se puede escribir como

$$ x(t) = x_0 + \int_{t_0}^t f(s, x) \,\textrm{d}s. $$

Esta relación, vista como un operador $(t_0, x_0)\rightarrow (t,x(t))$, se le conoce como
el operador de Picard-Lindelöf.

(b) Muestra que *iteraciones sucesivas* del operador de Picard-Lindelöf, obtenidas al ir truncando la serie de Taylor de $f(t,x)$ de manera consistente, generan la misma regla de recurrencia para la solución en términos de la serie de Taylor. Lo más sencillo es partir de la serie a orden $k$ de la solución de la ecuación diferencial, y obtener el siguiente término.

(c) Implementa una función que integre una serie (en una variable independiente); trata a la constante de integración (que sería la condición inicial en este caso) como un parámetro de la función.

(d) Construye la solución del ejercicio **2** usando *iteraciones* de Picard.